In [2]:
!pip install langchain-groq langchain

     |████████████████████████████████| 1.0 MB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 397 kB 29.6 MB/s eta 0:00:01
     |████████████████████████████████| 106 kB 39.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 20.9 MB/s eta 0:00:01
     |████████████████████████████████| 434 kB 39.1 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 7.2 MB/s eta 0:00:01
     |████████████████████████████████| 746 kB 35.5 MB/s eta 0:00:01
     |████████████████████████████████| 17.3 MB 40.6 MB/s eta 0:00:01
     |████████████████████████████████| 296 kB 50.2 MB/s eta 0:00:01
     |████████████████████████████████| 89 kB 22.2 MB/s eta 0:00:01
     |████████████████████████████████| 76 kB 14.6 MB/s eta 0:00:01
     |████████████████████████████████| 319 kB 52.9 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 5.2 MB/s  eta 0:00:01
     |████████████████████████████████| 129 kB 49.6 MB/s eta 0:00:01
     |████████████████████████████████

In [5]:
#Cell 1: Install Required Packages
!pip install langchain
!pip install langchain-groq

In [6]:
#Cell 2: Verify Installations
!pip list | grep langchain

langchain                0.2.16     
langchain-core           0.2.41     
langchain-groq           0.1.10     
langchain-text-splitters 0.2.4      


In [7]:
#Cell 3: Basic Setup
import warnings
import time
from datetime import datetime
warnings.filterwarnings('ignore')

from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

# Helper function for latency measurement
def measure_latency(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        latency = end_time - start_time
        print(f"Latency: {latency:.2f} seconds")
        return result
    return wrapper

In [8]:
#Cell 4: Set up Groq API
import os
# Replace with your actual Groq API key
os.environ["GROQ_API_KEY"] = "Type Your Groq API key"

# Initialize Groq LLM
llm = ChatGroq(
    temperature=0.9,
    model_name="mixtral-8x7b-32768"  # or "llama2-70b-4096"
)

In [9]:
#Cell 5: Basic LLMChain Implementation
@measure_latency
def test_llm_chain():
    print("\n=== Testing Basic LLMChain ===")
    
    # Create prompt template
    prompt = ChatPromptTemplate.from_template(
        "What is the best name to describe a company that makes {product}?"
    )
    
    # Create chain
    chain = LLMChain(llm=llm, prompt=prompt)
    
    # Test the chain
    product = "Queen Size Sheet Set"
    print(f"Input Product: {product}")
    print("Generating response...")
    result = chain.run(product)
    print(f"Generated company name: {result}")
    return result

# Run the test
result = test_llm_chain()


=== Testing Basic LLMChain ===
Input Product: Queen Size Sheet Set
Generating response...
Generated company name: Choosing a name for a company that makes queen size sheet sets can depend on various factors such as the brand's identity, target market, and values. However, here are some name suggestions that could work well for such a company:

1. Queen's Linen: This name suggests a focus on high-quality, luxurious linens specifically for queen-size beds.
2. The Sheet Society: This name implies a sense of exclusivity and sophistication, positioned as a members-only club for those who appreciate fine sheets.
3. Soft as a Cloud: This name conveys a sense of comfort and coziness, suggesting that the sheets will provide a restful night's sleep.
4. Sateen Dreams: This name focuses on the type of fabric used for the sheets, and the dream-like experience of sleeping on them.
5. Bedtime Bliss: This name suggests that the sheets will provide the ultimate relaxation and comfort, making bedtime a

In [10]:
#Cell 6: SimpleSequentialChain Implementation
@measure_latency
def test_simple_sequential_chain():
    print("\n=== Testing SimpleSequentialChain ===")
    
    # First chain - Company name generation
    first_prompt = ChatPromptTemplate.from_template(
        "What is the best name to describe a company that makes {product}?"
    )
    chain_one = LLMChain(llm=llm, prompt=first_prompt)
    
    # Second chain - Company description generation
    second_prompt = ChatPromptTemplate.from_template(
        "Write a 20 words description for the following company: {company_name}"
    )
    chain_two = LLMChain(llm=llm, prompt=second_prompt)
    
    # Combine chains
    overall_chain = SimpleSequentialChain(
        chains=[chain_one, chain_two],
        verbose=True
    )
    
    # Test the chain
    product = "Queen Size Sheet Set"
    print(f"\nInput Product: {product}")
    print("Generating response...")
    result = overall_chain.run(product)
    print(f"\nFinal Result: {result}")
    return result

# Run the test
result = test_simple_sequential_chain()


=== Testing SimpleSequentialChain ===

Input Product: Queen Size Sheet Set
Generating response...


> Entering new SimpleSequentialChain chain...
Here are a few suggestions for a company name that makes Queen Size Sheet Sets:

1. Queen Size Comfort Co.
2. Luxe Linen Co.
3. The Sheet Society
4. Soft Sleeper Co.
5. Bedtime Bliss Co.
6. The Queen's Quilt Co.
7. Royal Rest Co.
8. Dreamweaver Co.
9. The Slumber Company
10. Serene Sleep Co.

Of course, the best name for your company will depend on a number of factors, including your target market, brand positioning and personal preference. I would recommend brainstorming a list of words and phrases that evoke the qualities you want your brand to represent, and then playing around with different combinations until you find a name that feels right for your business. Good luck!
"Indulge in Luxurious Queen Size Sheet Sets from Serene Sleep Co."

> Finished chain.

Final Result: "Indulge in Luxurious Queen Size Sheet Sets from Serene Sleep Co."


In [11]:
#Cell 7: SequentialChain Implementation
@measure_latency
def test_sequential_chain():
    print("\n=== Testing SequentialChain ===")
    
    # Chain 1: Translation
    first_prompt = ChatPromptTemplate.from_template(
        "Translate the following review to english:\n\n{Review}"
    )
    chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                         output_key="English_Review")
    
    # Chain 2: Summarization
    second_prompt = ChatPromptTemplate.from_template(
        "Can you summarize the following review in 1 sentence:\n\n{English_Review}"
    )
    chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                         output_key="summary")
    
    # Chain 3: Language Detection
    third_prompt = ChatPromptTemplate.from_template(
        "What language is the following review:\n\n{Review}"
    )
    chain_three = LLMChain(llm=llm, prompt=third_prompt,
                          output_key="language")
    
    # Chain 4: Follow-up Message
    fourth_prompt = ChatPromptTemplate.from_template(
        "Write a follow up response to the following summary in the specified language:"
        "\n\nSummary: {summary}\n\nLanguage: {language}"
    )
    chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                         output_key="followup_message")
    
    # Combine all chains
    overall_chain = SequentialChain(
        chains=[chain_one, chain_two, chain_three, chain_four],
        input_variables=["Review"],
        output_variables=["English_Review", "summary", "followup_message"],
        verbose=True
    )
    
    # Test the chain
    review = "Ce produit est excellent!"
    print(f"\nInput Review: {review}")
    print("Processing through chains...")
    result = overall_chain(review)
    
    print("\nResults:")
    for key, value in result.items():
        print(f"{key}: {value}")
    return result

# Run the test
result = test_sequential_chain()


=== Testing SequentialChain ===

Input Review: Ce produit est excellent!
Processing through chains...


> Entering new SequentialChain chain...

> Finished chain.

Results:
Review: Ce produit est excellent!
English_Review: This product is excellent!
summary: The reviewer highly recommends the product, finding it to be of exceptional quality.
followup_message: Cher client,

Nous sommes ravis de lire que vous avez été séduit par la qualité exceptionnelle de notre produit. Nous travaillons dur pour garantir la satisfaction de nos clients et recevoir des commentaires aussi positifs est très gratifiant.

Nous sommes heureux d'apprendre que vous ayez trouvé notre produit "excellent" - ce mot nous touche droit au cœur. Nous continuerons à nous efforcer de maintenir ces normes élevées et d'offrir des produits de qualité supérieure à tous nos clients.

Merci beaucoup pour votre review élogieuse!

Cordialement,
[Votre nom]
Latency: 1.99 seconds


In [12]:
#Cell 8: Router Chain Templates
# Define templates for different domains
def get_prompt_templates():
    templates = {
        "physics": """You are a very smart physics professor. 
        You are great at answering questions about physics in a concise
        and easy to understand manner. 
        When you don't know the answer to a question you admit
        that you don't know.

        Here is a question:
        {input}""",
        
        "math": """You are a very good mathematician. 
        You are great at answering math questions. 
        You are so good because you are able to break down 
        hard problems into their component parts,
        answer the component parts, and then put them together
        to answer the broader question.

        Here is a question:
        {input}""",
        
        "history": """You are a very good historian. 
        You have an excellent knowledge of and understanding of people,
        events and contexts from a range of historical periods. 

        Here is a question:
        {input}""",
        
        "computer_science": """You are a successful computer scientist.
        You are great at answering coding questions and solving problems.

        Here is a question:
        {input}"""
    }
    return templates

In [15]:
#Cell 9: Router Chain Implementation and Testing
@measure_latency
def setup_and_test_router_chain():
    print("\n=== Testing RouterChain ===")
    
    # Define the prompt templates with clearer structure
    physics_template = """You are a physics expert. Answer this physics question: {input}"""
    math_template = """You are a math expert. Solve this math problem: {input}"""
    history_template = """You are a history expert. Answer this history question: {input}"""
    cs_template = """You are a computer science expert. Answer this CS question: {input}"""

    # Create destination chains dictionary directly
    destination_chains = {
        "physics": LLMChain(llm=llm, prompt=ChatPromptTemplate.from_template(physics_template)),
        "math": LLMChain(llm=llm, prompt=ChatPromptTemplate.from_template(math_template)),
        "history": LLMChain(llm=llm, prompt=ChatPromptTemplate.from_template(history_template)),
        "computer_science": LLMChain(llm=llm, prompt=ChatPromptTemplate.from_template(cs_template))
    }

    # Create default chain
    default_chain = LLMChain(
        llm=llm,
        prompt=ChatPromptTemplate.from_template("{input}")
    )

    # Define router template
    router_template = """Given a raw text input, route it to one of these categories:
    physics: Questions about physics concepts, theories, and phenomena
    math: Questions about mathematics, calculations, and mathematical concepts
    history: Questions about historical events, people, and periods
    computer_science: Questions about programming, algorithms, and computer concepts
    
    Input: {input}
    
    Return a JSON object formatted like:
    ```json
    {{"destination": "category_name", "next_inputs": "the original input"}}
    ```
    where category_name is one of [physics, math, history, computer_science]
    """

    # Create router chain
    router_chain = LLMRouterChain.from_llm(
        llm=llm, 
        prompt=PromptTemplate(
            template=router_template,
            input_variables=["input"],
            output_parser=RouterOutputParser()
        )
    )

    # Create multi-prompt chain
    chain = MultiPromptChain(
        router_chain=router_chain,
        destination_chains=destination_chains,
        default_chain=default_chain,
        verbose=True
    )

    # Test questions
    test_questions = [
        "What is black body radiation?",
        "What is 2 + 2?",
        "What is the significance of the Industrial Revolution?",
        "Explain what recursion is in programming"
    ]
    
    # Run tests
    for question in test_questions:
        try:
            print(f"\nQuestion: {question}")
            print("Generating response...")
            result = chain.run(question)
            print(f"Answer: {result}")
        except Exception as e:
            print(f"Error processing question: {e}")
        print("-" * 50)

    return chain

# Run the router chain test
router_chain = setup_and_test_router_chain()


=== Testing RouterChain ===

Question: What is black body radiation?
Generating response...


> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.
Answer: Black body radiation is the electromagnetic radiation emitted by a perfect absorber or emitter of electromagnetic radiation, known as a black body. The concept of a black body is an idealization, as perfect absorbers and emitters do not exist in nature. However, objects such as stars and hot furnaces can approximate black body behavior over a narrow range of wavelengths.

The spectrum of black body radiation is continuous and depends only on the temperature of the black body. It is described by Planck's law, which gives the intensity of radiation per unit area, per unit solid angle, and per unit frequency interval as:

I(ν,T) = (2hν^3/c^2) / (exp(hν/kT) - 1)

where I(ν,T) is the intensity, ν is the frequency, T is the temperature, h is Planck's constant, c is the speed of lig

In [16]:
def run_all_tests():
    print("\n========= Running All Chain Tests =========\n")
    
    # Test LLMChain
    print("\n1. Testing LLMChain:")
    test_llm_chain()
    
    # Test SimpleSequentialChain
    print("\n2. Testing SimpleSequentialChain:")
    test_simple_sequential_chain()
    
    # Test SequentialChain
    print("\n3. Testing SequentialChain:")
    test_sequential_chain()
    
    # Router chain was already tested, but we can use the existing instance
    print("\n4. Testing RouterChain with additional questions:")
    if 'router_chain' in globals():
        questions = [
            "How does photosynthesis work?",
            "What is the square root of 16?",
            "Who was Alexander the Great?"
        ]
        for question in questions:
            try:
                print(f"\nQuestion: {question}")
                result = router_chain.run(question)
                print(f"Answer: {result}")
            except Exception as e:
                print(f"Error: {e}")
            print("-" * 50)

# Run all tests
run_all_tests()


========= Running All Chain Tests =========


1. Testing LLMChain:

=== Testing Basic LLMChain ===
Input Product: Queen Size Sheet Set
Generating response...
Generated company name: Choosing a name for a company can be a fun and exciting task! When considering a name for a company that makes Queen Size Sheet Sets, you may want to think about words that evoke a sense of comfort, luxury, and high quality. Here are a few suggestions:

1. Queen's Comfort: This name suggests that the company's sheet sets are designed with the ultimate comfort of the queen-size sleeper in mind.
2. Luxe Linens: This name conveys a sense of luxury and sophistication, suggesting that the company's sheet sets are made from the finest materials.
3. Royal Slumber: This name suggests that the company's sheet sets are fit for a queen, and are designed to promote deep, restful sleep.
4. Plush Palace: This name suggests that the company's sheet sets are plush and indulgent, creating a luxurious sleeping experience.
5